In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, confusion_matrix


file_path = 'smote_enn_balanced_dataset.csv'
df = pd.read_csv(file_path)


X = df.drop('Diabetes_binary', axis=1)
y = df['Diabetes_binary']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "Stacking": StackingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('adaboost', AdaBoostClassifier(n_estimators=100, random_state=42)),
            ('gradient_boosting', GradientBoostingClassifier(n_estimators=100, random_state=42))
        ],
        final_estimator=LogisticRegression(),
        passthrough=True
    )
}

# Train and evaluate each classifier
results = {}
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)[:, 1]  # probabilities for the positive class

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)  # or use average='binary' for binary classification
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_proba)
    
    # Calculate specificity from the confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)

    # Store results
    results[name] = {
        'Accuracy': accuracy,
        'Sensitivity (Recall)': recall,
        'Precision': precision,
        'F1 Score': f1,
        'AUC-ROC': auc_roc
    }

# Print the results
for name, metrics in results.items():
    print(f"\n{name} Performance:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

/Users/tietie/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Random Forest Performance:
Accuracy: 0.96
Sensitivity (Recall): 0.96
Precision: 0.96
F1 Score: 0.96
AUC-ROC: 0.99

AdaBoost Performance:
Accuracy: 0.88
Sensitivity (Recall): 0.87
Precision: 0.88
F1 Score: 0.88
AUC-ROC: 0.95

Gradient Boosting Performance:
Accuracy: 0.89
Sensitivity (Recall): 0.89
Precision: 0.89
F1 Score: 0.89
AUC-ROC: 0.96

Stacking Performance:
Accuracy: 0.97
Sensitivity (Recall): 0.96
Precision: 0.96
F1 Score: 0.96
AUC-ROC: 0.99
